In [1]:
// 📌𝐓𝐢𝐠𝐞𝐫 𝐀𝐧𝐚𝐥𝐲𝐭𝐢𝐜𝐬 𝐒𝐐𝐋 𝐈𝐧𝐭𝐞𝐫𝐯𝐢𝐞𝐰 𝐐𝐮𝐞𝐬𝐭𝐢𝐨𝐧

// Here is today's SQL question 

// ❓Find the count of new customers added in each Month


import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._


val data = Seq(
  Row("2021-01-01", "C1", 20),
  Row("2021-01-01", "C2", 30),
  Row("2021-02-01", "C1", 10),
  Row("2021-02-01", "C3", 15),
  Row("2021-03-01", "C5", 19),
  Row("2021-03-01", "C4", 10),
  Row("2021-04-01", "C3", 13),
  Row("2021-04-01", "C5", 15),
  Row("2021-04-01", "C6", 10)
)

val schema = StructType(Array(
  StructField("order_date", StringType),
  StructField("customer", StringType),
  StructField("qty", IntegerType)
))
      
val rdd = spark.sparkContext.parallelize(data)
val df = spark.createDataFrame(rdd, schema)

                        
df.show(false)

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.102:4040
SparkContext available as 'sc' (version = 3.4.1, master = local[*], app id = local-1704624295653)
SparkSession available as 'spark'


+----------+--------+---+
|order_date|customer|qty|
+----------+--------+---+
|2021-01-01|C1      |20 |
|2021-01-01|C2      |30 |
|2021-02-01|C1      |10 |
|2021-02-01|C3      |15 |
|2021-03-01|C5      |19 |
|2021-03-01|C4      |10 |
|2021-04-01|C3      |13 |
|2021-04-01|C5      |15 |
|2021-04-01|C6      |10 |
+----------+--------+---+



import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._
data: Seq[org.apache.spark.sql.Row] = List([2021-01-01,C1,20], [2021-01-01,C2,30], [2021-02-01,C1,10], [2021-02-01,C3,15], [2021-03-01,C5,19], [2021-03-01,C4,10], [2021-04-01,C3,13], [2021-04-01,C5,15], [2021-04-01,C6,10])
schema: org.apache.spark.sql.types.StructType = StructType(StructField(order_date,StringType,true),StructField(customer,StringType,true),StructField(qty,IntegerType,true))
rdd: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = ParallelCollectionRDD[0] at parallelize at <console>:53
df: org.apache.spark.sql.DataFrame = [order_date: string, customer: string ... 1 more field]


In [10]:
import org.apache.spark.sql.expressions.Window

val df1 = df.withColumn("order_date", to_date($"order_date", "yyyy-MM-dd")
             ).withColumn("rn", row_number().over(Window.partitionBy($"customer").orderBy($"order_date"))
                         )

df1.show(false)


df1.filter($"rn" === 1
          ).withColumn("month_name", date_format($"order_date", "MMMM")
                      ).groupBy($"month_name", $"order_date").count().show(false)

+----------+--------+---+---+
|order_date|customer|qty|rn |
+----------+--------+---+---+
|2021-01-01|C1      |20 |1  |
|2021-02-01|C1      |10 |2  |
|2021-01-01|C2      |30 |1  |
|2021-02-01|C3      |15 |1  |
|2021-04-01|C3      |13 |2  |
|2021-03-01|C4      |10 |1  |
|2021-03-01|C5      |19 |1  |
|2021-04-01|C5      |15 |2  |
|2021-04-01|C6      |10 |1  |
+----------+--------+---+---+

+----------+----------+-----+
|month_name|order_date|count|
+----------+----------+-----+
|April     |2021-04-01|1    |
|February  |2021-02-01|1    |
|January   |2021-01-01|2    |
|March     |2021-03-01|2    |
+----------+----------+-----+



import org.apache.spark.sql.expressions.Window
df1: org.apache.spark.sql.DataFrame = [order_date: date, customer: string ... 2 more fields]
